In [1]:
# Homework 4
# Center of Mass Position and Velocity
# Malhar Dave

In [2]:
# remember this is just a template, you don't need to follow every step
# if you have your own method to solve the homework, it is totally fine

In [3]:
# import modules
import numpy as np
import astropy.units as u
import astropy.table as tbl

from ReadFile import Read

In [4]:
class CenterOfMass:
# Class to define COM position and velocity properties of a given galaxy 
# and simulation snapshot

    def __init__(self, filename, ptype):
        ''' Class to calculate the 6-D phase-space position of a galaxy's center of mass using
        a specified particle type. 
            
            PARAMETERS
            ----------
            filename : `str`
                snapshot file
            ptype : `int; 1, 2, or 3`
                particle type to use for COM calculations
        '''
     
        # read data in the given file using Read
        self.time, self.total, self.data = Read(filename)                                                                                             

        #create an array to store indexes of particles of desired Ptype                                
        self.index = np.where(self.data['type'] == ptype)

        # store the mass, positions, velocities of only the particles of the given type
        # the following only gives the example of storing the mass
        self.m = self.data['m'][self.index]
        # write your own code to complete this for positions and velocities
        self.x = self.data['x'][self.index]
        self.y = self.data['y'][self.index]
        self.z = self.data['z'][self.index]
        self.vx = self.data['vx'][self.index] 
        self.vy = self.data['vy'][self.index]
        self.vz = self.data['vz'][self.index]


    def COMdefine(self,a,b,c,m):
        ''' Method to compute the COM of a generic vector quantity by direct weighted averaging.
        
        PARAMETERS
        ----------
        a : `float or np.ndarray of floats`
            first vector component
        b : `float or np.ndarray of floats`
            second vector component
        c : `float or np.ndarray of floats`
            third vector component
        m : `float or np.ndarray of floats`
            particle masses
        
        RETURNS
        -------
        a_com : `float`
            first component on the COM vector
        b_com : `float`
            second component on the COM vector
        c_com : `float`
            third component on the COM vector
        '''
        # write your own code to compute the generic COM 
        #using Eq. 1 in the homework instructions
        # xcomponent Center of mass
        a_com = np.sum((a*m)/np.sum(m))
        # ycomponent Center of mass
        b_com =  np.sum((b*m)/np.sum(m))
        # zcomponent Center of mass
        c_com =  np.sum((c*m)/np.sum(m))
        return a_com,b_com,c_com
    
        
        # return the 3 components separately
       
    
    
    def COM_P(self, delta,volDec):
        '''Method to compute the position of the center of mass of the galaxy 
        using the shrinking-sphere method.

        PARAMETERS
        ----------
        delta : `float, optional`
            error tolerance in kpc. Default is 0.1 kpc
        
        RETURNS
        ----------
        p_COM : `np.ndarray of astropy.Quantity'
            3-D position of the center of mass in kpc
        '''                                                                     

        # Center of Mass Position                                                                                      
        ###########################                                                                                    

        # Try a first guess at the COM position by calling COMdefine                                                   
        x_COM, y_COM, z_COM = self.COMdefine(self.x, self.y, self.z, self.m)
        # compute the magnitude of the COM position vector.
        # write your own code below
        r_COM = np.sqrt(x_COM**2+y_COM**2+z_COM**2)


        # iterative process to determine the center of mass                                                            

        # change reference frame to COM frame                                                                          
        # compute the difference between particle coordinates                                                          
        # and the first guess at COM position
        # write your own code below
        x_new = self.x - x_COM 
        y_new = self.y - y_COM
        z_new = self.z -z_COM
        r_new = np.sqrt(x_new**2+y_new**2+z_new**2) 
        print(r_new)
        
        # find the max 3D distance of all particles from the guessed COM                                               
        # will re-start at half that radius (reduced radius)                                                           
        r_max = max(r_new)/volDec
        print(r_max)
        # pick an initial value for the change in COM position                                                      
        # between the first guess above and the new one computed from half that volume
        # it should be larger than the input tolerance (delta) initially
        change = 1000.0

        # start iterative process to determine center of mass position                                                 
        # delta is the tolerance for the difference in the old COM and the new one.    
        
        while ((change > delta)):
            # select all particles within the reduced radius (starting from original x,y,z, m)
            # write your own code below (hints, use np.where)
            index2 = np.where(r_new<r_max)
            print(index2)
            x2 = self.x[index2]
            y2 = self.y[index2]
            z2 = self.z[index2]
            m2 = self.m[index2]
            
            # Refined COM position:                                                                                    
            # compute the center of mass position using                                                                
            # the particles in the reduced radius
            # write your own code below
            #x_COM2, y_COM2, z_COM2 = self.COMdefine(self.x2, self.y2, self.z2, self.m2)
            x_COM2, y_COM2, z_COM2 = self.COMdefine(x2, y2, z2, m2)
            # compute the new 3D COM position
            # write your own code below
            r_COM2 = np.sqrt(x_COM2**2+y_COM2**2+z_COM2**2)

            # determine the difference between the previous center of mass position                                    
            # and the new one.                                                                                         
            change = np.abs(r_COM - r_COM2)
            # uncomment the following line if you want to check this   
            print(r_COM)
            print(r_COM2)
            print ("CHANGE = ", change)                                                                                     

            # Before loop continues, reset : r_max, particle separations and COM                                        

            # reduce the volume by a factor of 2 again                                                                 
            r_max /= 2.0
            # check this.                                                                                              
            print ("maxR", r_max)                                                                                      

            # Change the frame of reference to the newly computed COM.                                                 
            # subtract the new COM
            # write your own code below
            x_new = self.x-x_COM2
            y_new = self.y-y_COM2
            z_new = self.z-z_COM2
            r_new = np.sqrt(x_new**2+y_new**2+z_new**2)
                     
            # set the center of mass positions to the refined values                                                   
            x_COM = x_COM2
            y_COM = y_COM2
            z_COM = z_COM2
            r_COM = r_COM2

            # create an array (np.array) to store the COM position                                                                                                                                                       
            p_COM = np.array([x_COM, y_COM, z_COM]) 
        return np.around(p_COM, 2)*u.kpc
        
        
        
        
    def COM_V(self, x_COM, y_COM, z_COM):
        ''' Method to compute the center of mass velocity based on the center of mass
        position.

        PARAMETERS
        ----------
        x_COM : 'astropy quantity'
            The x component of the center of mass in kpc
        y_COM : 'astropy quantity'
            The y component of the center of mass in kpc
        z_COM : 'astropy quantity'
            The z component of the center of mass in kpc
            
        RETURNS
        -------
        v_COM : `np.ndarray of astropy.Quantity'
            3-D velocity of the center of mass in km/s
        '''
        
        # the max distance from the center that we will use to determine 
        #the center of mass velocity                   
        rv_max = (15.0)*u.kpc
        
        
        
        # determine the position of all particles relative to the center of mass position (x_COM, y_COM, z_COM)
        # write your own code below
        
        
       
        # determine the index for those particles within the max radius
        # write your own code below
        
        
        # determine the velocity and mass of those particles within the mas radius
        # write your own code below
        xV = self.x[:]*u.kpc - x_COM
        yV = self.y[:]*u.kpc - y_COM
        zV = self.z[:]*u.kpc - z_COM
        rV = np.sqrt(xV**2 + yV**2 + zV**2)
        
        indexV = np.where(rV < rv_max)
        
        # determine the velocity and mass of those particles within the mas radius
        # write your own code below
        vx_new = self.vx[indexV]
        vy_new = self.vy[indexV]
        vz_new = self.vz[indexV]
        m_new =  self.m[indexV]
       
        
        # compute the center of mass velocity using those particles
        # write your own code below
        vx_COM, vy_COM, vz_COM =   self.COMdefine(vx_new,vy_new,vz_new, m_new)
        
        # create an array to store the COM velocity
        # write your own code below
        v_COM = np.array([vx_COM,vy_COM,vz_COM])

        # return the COM vector
        # set the correct units using astropy
        # round all values
        return np.round(v_COM, 2)*u.km/u.s
    

In [5]:
# Create a Center of mass object for the MW, M31 and M33
# below is an example of using the class for MW
MW_COM = CenterOfMass("MW_000.txt", 2)
#now we use the class for M31
M31_COM = CenterOfMass("M31_000.txt", 2)
#now we use the class for M33
M33_COM = CenterOfMass("M33_000.txt", 2)

In [6]:
# below gives you an example of calling the class's functions
# MW:   store the position and velocity COM

MW_COM_p = MW_COM.COM_P(0.1) 
print(MW_COM_p)
MW_COM_v = MW_COM.COM_V(MW_COM_p[0], MW_COM_p[1], MW_COM_p[2]) 
print(MW_COM_v)

# M31:   store the position and velocity COM for M31
M31_COM_p = M31_COM.COM_P(0.1) 
print(M31_COM_p) 
M31_COM_v = M31_COM.COM_V(M31_COM_p[0], M31_COM_p[1], M31_COM_p[2]) 
print(M31_COM_v)

# M33:   store the position and velocity COM for M33
M33_COM_p = M33_COM.COM_P(0.1) 
print(M33_COM_p) 
M33_COM_v = M33_COM.COM_V(M33_COM_p[0], M33_COM_p[1], M33_COM_p[2]) 
print(M33_COM_v)


[16.06367617  5.82531863  2.08451827 ...  3.83157401  7.27104355
  0.9380769 ]
22.040962666796222
(array([    0,     1,     2, ..., 74997, 74998, 74999], dtype=int64),)
2.950193027732071
3.1799740718728198
CHANGE =  0.22978104414074885
maxR 11.020481333398111
(array([    1,     2,     3, ..., 74997, 74998, 74999], dtype=int64),)
3.1799740718728198
3.6504077061113
CHANGE =  0.4704336342384803
maxR 5.5102406666990555
(array([    2,     6,     7, ..., 74994, 74997, 74999], dtype=int64),)
3.6504077061113
3.8512980828560566
CHANGE =  0.20089037674475652
maxR 2.7551203333495278
(array([    2,     7,     9, ..., 74993, 74994, 74999], dtype=int64),)
3.8512980828560566
3.881485248053005
CHANGE =  0.030187165196948307
maxR 1.3775601666747639
[-2.07  2.95 -1.45] kpc
[ 0.94  6.32 -1.35] km / s
[ 9.12675349  1.07492378  2.6751642  ...  0.76792937 34.89327081
 10.85223558]
33.36459887755717
(array([     0,      1,      2, ..., 119996, 119997, 119999], dtype=int64),)
773.0189990729882
772.97440477618

In [7]:
# now write your own code to answer questions

In [8]:
#Now we print our answers for questions 1 to 4 and write them down 
#QUESTION 1: 
print("COM position(in kpc) of MW is:",np.around((MW_COM_p),3)*u.kpc)  #[-0.93398577  2.40892628 -1.42421335] kpc
print("COM velocity (in km/s) of MW is:",np.around((MW_COM_v),3)*u.km/u.s) # 2.95 km/s 
print("COM position(in kpc) of M31 is:",np.around((M31_COM_p),3)*u.kpc)# [-377.72135607  611.45569706 -284.6170772 ] kpc
print("COM velocity (in km/s) of M31 is:",np.around((M31_COM_v),3)*u.km/u.s) #773.02 km/s
print("COM position(in kpc) of M33 is:",np.around((M33_COM_p),3)*u.kpc) # [-476.24732441  491.45325118 -412.38026398] kpc
print("COM velocity (in km/s) of M33 is:",np.around((M33_COM_v),3)*u.km/u.s) #799.0  km/s

#Question 2: magnitude between andromeda and milky way is : 
x=np.abs((MW_COM_p)-(M31_COM_p))
print("position magnitude between andromeda and milky way:", (np.around(x,3))*u.kpc)
#[376.78737029 609.04677078 283.19286385] kpc
y=np.abs((MW_COM_v)-(M31_COM_v))
print("velocity magnitude between andromeda and milky way:",np.around(y,3)*u.km/u.s)
#770.0699999999999 km/s

#Question 3: print("position magnitude between andromeda and M33:", 
x2=(np.abs((M31_COM_p)-(M33_COM_p)))
y2=(np.abs((M31_COM_v)-(M33_COM_v)))
print("position magnitude between andromeda and M33:", np.around(x2,3)*u.kpc) #[ 98.526 120.002 127.763] kpc
print("velocity magnitude between andromeda and M33:",np.around(y2,3)*u.km/u.s) # 25.98 km / s

#Question 4: An interative process is good for eliminating outliers which will 
#outdoubtedly emerge as some stars will be flung extremelly far away from the merging galaxies 
#and negatively affect data and make it a poor measure of actual COM during the merger 

COM position(in kpc) of MW is: [-2.07  2.95 -1.45] kpc2
COM velocity (in km/s) of MW is: [ 0.94  6.32 -1.35] km2 / s2
COM position(in kpc) of M31 is: [-377.66  611.43 -284.64] kpc2
COM velocity (in km/s) of M31 is: [ 72.85 -72.14  49.  ] km2 / s2
COM position(in kpc) of M33 is: [-476.22  491.44 -412.4 ] kpc2
COM velocity (in km/s) of M33 is: [ 44.42 101.78 142.23] km2 / s2
position magnitude between andromeda and milky way: [375.59 608.48 283.19] kpc2
velocity magnitude between andromeda and milky way: [71.91 78.46 50.35] km2 / s2
position magnitude between andromeda and M33: [ 98.56 119.99 127.76] kpc2
velocity magnitude between andromeda and M33: [ 28.43 173.92  93.23] km2 / s2
